<a href="https://colab.research.google.com/github/unt-iialab/BERT-Relation-Extraction/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,13,up all night
1,13,insomnia
2,60,cigarette withdrawal
3,85,tremor
4,121,itches


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-9.test.csv",header=None)
test.head()

,0,1
0,62,feeling proper psycho
1,74,useless without
2,85,tremor
3,91,jitters
4,97,screwing with my short term memory


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'drug',
 'by',
 'on',
 'patients',
 'that',
 'is',
 'as',
 'or',
 'at',
 'mg',
 'increased',
 'group',
 'from',
 'mood',
 'study',
 'treatment',
 'this',
 'after',
 'release',
 'p',
 'an',
 'disorder',
 'me',
 'be',
 ',',
 'i',
 'effect',
 'abnormal',
 'withdrawal',
 'decreased',
 'not',
 'than',
 'weight',
 'compared',
 'anxiety',
 'my',
 'groups',
 'no',
 'are',
 'using',
 'inflammatory',
 'symptoms',
 'effects',
 'two',
 'both',
 'significantly',
 'drugs',
 'acute',
 'significant',
 'potassium',
 'anti',
 'skin',
 'h',
 'between',
 'day',
 'dose',
 'disorders',
 'behavior',
 'used',
 'syndrome',
 'all',
 'voltaren',
 '(',
 ')',
 'blood',
 'time',
 'results',
 'had',
 'these',
 'placebo',
 'which',
 '-',
 'efficacy',
 'diseases',
 'have',
 'it',
 'concentration',
 'oral',
 'days',
 's

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.836923,4.222463,0.343939,17:25


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.024521,3.824425,0.380718,17:21


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.726233,3.780058,0.384856,17:25


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.676333,3.746818,0.388273,17:23


In [25]:
learn.save_encoder('TwADR_pubmed_fold9_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.245729,6.386784,0.100000,00:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.276859,5.085742,0.183333,00:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.740279,4.698279,0.200000,01:08


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.428067,4.323278,0.291667,01:09
1,5.193270,4.186599,0.308333,01:11


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.944137,4.038934,0.325000,01:06
1,4.735382,3.962600,0.333333,01:05


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.523766,3.832071,0.350000,01:06
1,4.392803,3.699164,0.325000,01:08


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.173926,3.641449,0.358333,01:06
1,4.037714,3.604354,0.375000,01:08


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.860774,3.599500,0.366667,01:07
1,3.742316,3.526067,0.400000,01:09


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.573823,3.507028,0.400000,01:11
1,3.479196,3.466616,0.375000,01:11
2,3.297896,3.346793,0.408333,01:07
3,3.172353,3.335024,0.408333,01:05


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.963335,3.327415,0.433333,01:07
1,2.889599,3.393071,0.400000,01:06
2,2.723500,3.320861,0.416667,01:08
3,2.579089,3.296616,0.425000,01:10


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.426032,3.263316,0.450000,01:15
1,2.381757,3.307495,0.425000,01:07
2,2.223342,3.313095,0.400000,01:11
3,2.068566,3.236233,0.425000,01:11


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.913295,3.317721,0.408333,01:06
1,1.890129,3.353286,0.383333,01:11
2,1.738014,3.328673,0.450000,01:09
3,1.598168,3.304400,0.450000,01:10


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.493916,3.442162,0.441667,01:11
1,1.513849,3.598176,0.450000,01:12
2,1.389174,3.550624,0.425000,01:07
3,1.278431,3.541938,0.408333,01:11


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.157450,3.594868,0.425000,01:05
1,1.177469,3.673963,0.425000,01:06
2,1.106714,3.740988,0.408333,01:08
3,0.998208,3.734622,0.408333,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.891396,3.700220,0.433333,01:10
1,0.926486,3.753081,0.425000,01:10
2,0.894709,3.804236,0.425000,01:08
3,0.803308,3.686432,0.425000,01:08


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.753685,3.764556,0.425000,01:09
1,0.726087,3.907835,0.433333,01:06


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.742977,3.937249,0.391667,01:07
1,0.686266,3.943910,0.391667,01:09


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.638621,3.973651,0.433333,01:12


In [44]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold9')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.640733,4.090849,0.433333,01:15


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.563254,3.980119,0.408333,01:15


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.538554,4.182415,0.425000,01:10


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[1529, 947, 1204, 2149, 1483, 189, 189, 189, 1099, 224, 224, 2081, 2147, 1936, 233, 2147, 249, 1096, 301, 561, 749, 1099, 1769, 749, 749, 749, 1925, 791, 1054, 792, 154, 2149, 1308, 978, 1884, 249, 1884, 852, 2023, 1096, 1136, 974, 974, 974, 974, 109, 978, 978, 989, 1019, 1529, 1062, 1718, 1096, 1526, 2032, 1925, 1136, 1136, 1594, 1798, 1458, 1340, 1529, 1096, 852, 265, 1099, 1597, 1822, 1676, 2149, 1529, 1432, 1432, 825, 1507, 1526, 1508, 1595, 1597, 1597, 1597, 1597, 1676, 249, 1942, 1610, 1610, 1526, 1701, 1526, 1701, 1526, 1711, 1711, 1727, 1727, 1767, 1099, 1767, 1727, 1669, 276, 1787, 1767, 1767, 1787, 1787, 1787, 1787, 1787, 2194, 1718, 1727, 670, 825, 1062, 1099, 1529, 1942, 2149, 2024, 2032, 2032, 1787, 1787, 1769, 2103, 813, 2147, 2147, 2147, 2147, 2149, 2103, 1340, 2194, 1529, 824, 2194, 2194]
[62, 74, 85, 91, 97, 189, 189, 189, 224, 224, 224, 233, 233, 233, 233, 233, 249, 250, 301, 504, 561, 620, 739, 749, 749, 749, 791, 791, 791, 792, 813, 818, 822, 822, 824, 824, 

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

59
0.4154929577464789
